In [ ]:
%%time
%matplotlib inline
import importlib
import new_import  
importlib.reload(new_import)
from new_import import *

In [ ]:
cluster, client = notebook_utils.initialize_dask(use_gateway=True, workers=(1,10))
# Khai báo 1 Datacube là dc
dc = datacube.Datacube()
client

In [ ]:
date_range = ('2022-09-01', '2023-10-01')
longtitude_range = (105.5, 106.4)
latitude_range = (9.2, 10.0) 

coordinates = (longtitude_range, latitude_range)

In [ ]:
dsvh, dsvv = load_data_sen1(dc, date_range, coordinates)

In [ ]:
average_vv = calculate_average(dsvv, time_pattern='1M')
progress(average_vv)

In [ ]:
average_vv.compute()

In [ ]:
coord_time = np.arange(13)
coord_x = average_vv.coords['x']
coord_y = average_vv.coords['y']

In [ ]:
full_dataset = np.load('input/saved_datasets/gb_full_dataset_filled.npz')
ndvi = full_dataset['ndvi']
vh = full_dataset['vh']
vv = full_dataset['vv']

In [ ]:
ndvi_xarray = xr.DataArray(ndvi, dims=["time", "y", "x"], coords={'time': coord_time, 'x': coord_x, 'y': coord_y})
vh_xarray = xr.DataArray(vh, dims=["time", "y", "x"], coords={'time': coord_time, 'x': coord_x, 'y': coord_y})
vv_xarray = xr.DataArray(vv, dims=["time", "y", "x"], coords={'time': coord_time, 'x': coord_x, 'y': coord_y})

In [ ]:
train_path = "input/train_points/1130/ST_1130_points.shp"  

# cấu hình nhãn dữ liệu 
label_mapping = {
    "Lua tom": "0",
    "Lua": "1",
    "CHN": "2",
    "CLN": "3",
    "TS": "4",
    "Song": "5",
    "Dat xay dung": "6",
    "Rung": "7",
}

labels = load_data_geo(train_path)
labels.head()

In [ ]:
# Extract data points and create the dataset
data_points = extract_data_point(labels, ndvi_xarray, vh_xarray, vv_xarray)
X_train, y_train = create_dataset(data_points)

In [ ]:
 X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
%%time
best_model, best_params, best_score, cv_results = find_best_classifier(X_train, 
                                                                       y_train, classifiers['naive_bayes'], 
                                                                       param_grids_classifier['naive_bayes'])

# Print best parameters, best result, and MSE over splits
print(f"Best parameters: {best_params}")
print(f"Best result : {best_score}")

In [ ]:
y_pred = best_model.predict(X_test)

# Calculate Mean Squared Error
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc}")

In [ ]:
folder_path = 'output/classifiers/'
file_path = os.path.join(folder_path, 'nb_classify.pkl')
joblib.dump(best_model, file_path)

In [ ]:
client.close()
cluster.close()